In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load pre-trained ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers for classification
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: partially damaged, non-damaged, collapsed
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Create ImageDataGenerator for training
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

# Load training data using ImageDataGenerator
train_generator = train_datagen.flow_from_directory(
    'dataset/train/',  # Path to your training dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Use categorical because there are multiple classes
)

# Train the model (No validation data)
model.fit(train_generator, epochs=10)

# Save the trained model
model.save('models/building_damage_classifier.h5')

# Example of using the model for prediction on a new image
def classify_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Predict the class
    predictions = model.predict(img_array)
    class_labels = ['partially damaged', 'non-damaged', 'collapsed']
    predicted_class = class_labels[np.argmax(predictions)]

    print(f"The building is classified as: {predicted_class}")

# Example use
classify_image('wall.jpg')


Found 9 images belonging to 3 classes.
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.1871 - accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 0s 342ms/step - loss: 1.8671 - accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 0s 339ms/step - loss: 1.2415 - accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 0s 343ms/step - loss: 0.8069 - accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 340ms/step - loss: 1.1322 - accuracy: 0.5556
Epoch 6/10
1/1 [==============================] - 0s 338ms/step - loss: 0.9798 - accuracy: 0.4444
Epoch 7/10
1/1 [==============================] - 0s 346ms/step - loss: 0.5920 - accuracy: 0.8889
Epoch 8/10
1/1 [==============================] - 0s 338ms/step - loss: 0.7855 - accuracy: 0.4444
Epoch 9/10
1/1 [==============================] - 0s 341ms/step - loss: 0.6075 - accuracy: 0.6667
Epoch 10/10
1/1 [==============================] - 1s 720ms/step
The building is c

In [4]:
print(tf.__version__)  # Verify TensorFlow version
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("All imports are successful!")

2.15.0
All imports are successful!
